In [205]:
!pip install gensim --user
!pip install textblob
!pip install scikit-learn --user
!pip install sklearn --user

In [206]:
!python --version

Python 3.8.5


In [207]:
!pip install scikit-learn==1.0.1
sklearn.__version__

ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\scikit_learn-0.23.2.dist-info\\COPYING'
Consider using the `--user` option or check the permissions.



'0.23.2'


  Using cached scikit_learn-1.0.1-cp38-cp38-win_amd64.whl (7.2 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.23.2
    Uninstalling scikit-learn-0.23.2:


In [15]:
import pandas as pd
import numpy as np
import re

#Timing
from tqdm import tqdm

#Word Embeddings
import nltk
from textblob import TextBlob
from gensim.models import Word2Vec
from gensim.parsing.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

#Models
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Cameron
[nltk_data]     Milne\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Cameron
[nltk_data]     Milne\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## EDA

In [22]:
reddit_comments = pd.read_csv('AnnotatedData/RC_2017-01_scored.csv')

In [23]:
scored_tweets = reddit_comments[['author', 'subreddit', 'body', 'toxicity', 'profanity', 'sexually_explicit']]

In [25]:
scored_tweets.rename(columns={'toxicity': 'TOXICITY', 'profanity': 'PROFANITY', 'sexually_explicit': 'SEXUALLY_EXPLICIT'}, inplace=True)

C:\Users\Cameron Milne\AppData\Roaming\Python\Python38\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [26]:
scored_tweets.columns

Index(['author', 'subreddit', 'body', 'TOXICITY', 'PROFANITY',
       'SEXUALLY_EXPLICIT'],
      dtype='object')

In [27]:
scored_tweets.corr()

,TOXICITY,PROFANITY,SEXUALLY_EXPLICIT
TOXICITY,1.000000,0.962363,0.655689
PROFANITY,0.962363,1.000000,0.659925
SEXUALLY_EXPLICIT,0.655689,0.659925,1.000000


In [28]:
scored_tweets.head(3)

,author,subreddit,body,TOXICITY,PROFANITY,SEXUALLY_EXPLICIT
0,Frantic_Mantid,awwnverts,Those eyes! I don't think I've ever seen a spi...,0.203826,0.126241,0.257775
1,I_LICK_PUPPIES,longboarding,I didn't even catch on... Lmao god damn it.,0.837652,0.919330,0.202414
2,PCMRwill0956,pcmasterrace,I'll take it\n,0.058286,0.029915,0.102896


In [29]:
scored_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38406 entries, 0 to 38405
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   author             38406 non-null  object 
 1   subreddit          38406 non-null  object 
 2   body               38406 non-null  object 
 3   TOXICITY           38367 non-null  float64
 4   PROFANITY          38367 non-null  float64
 5   SEXUALLY_EXPLICIT  38367 non-null  float64
dtypes: float64(3), object(3)
memory usage: 1.8+ MB


## New Preprocessing

In [30]:
scored_tweets.dropna(subset=['body'], inplace=True)

C:\Users\Cameron Milne\AppData\Roaming\Python\Python38\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [31]:
p = PorterStemmer()

In [32]:
def preprocessing(tweets_array):
    
    """
    Take in an array of tweets, and return the processed tweets.
    remove stopwords, usernames, punctuation, and words with length of 1
    """
    processed_array = []
    
    for tweet in tqdm(tweets_array):
        processed_punc = re.sub('[^@a-zA-Z ]', '', tweet)  # remove punctuation
        processed = re.sub('https\S+|@\S+|\b\w{,1}\b', '', processed_punc)  # remove https, usernames, len(word) < 1
        words = word_tokenize(processed)
        tokens_without_sw = [word for word in words if not word in stopwords.words()]
        sentence = ' '.join(tokens_without_sw)
        stemmed_sentence = p.stem_sentence(sentence)
        processed_array.append(stemmed_sentence)
    
    return processed_array

In [34]:
scored_tweets['processed'] = preprocessing(scored_tweets['body'])

100%|██████████| 100/100 [00:31<00:00,  3.16it/s]


In [217]:
# from nltk.tokenize import sent_tokenize, word_tokenize

# #tokenizes every tweet
# tokenizer = nltk.RegexpTokenizer(r"\w+")
# scored_tweets['processed'] = scored_tweets['text'].apply(tokenizer.tokenize)
# scored_tweets['processed']

In [218]:
# from nltk.corpus import stopwords 

# #Widely used stopword library
# stop_words = set(stopwords.words('english'))

# #gets rid of numbers
# stop_words.update([str(x) for x in np.arange(10)])

# #apply to tokens
# scored_tweets['processed'] = scored_tweets.processed.apply(lambda x: [i for i in x if not i in stop_words])
# scored_tweets['processed']

In [219]:
# scored_tweets['processed'] = [' '.join(l) for l in scored_tweets['processed']]
# scored_tweets['processed']

In [35]:
#Convert Scores to Round Numbers
scored_tweets['TOXICITY'] = round(scored_tweets['TOXICITY'] * 10, 0)
scored_tweets['PROFANITY'] = round(scored_tweets['PROFANITY'] * 10, 0)
scored_tweets['SEXUALLY_EXPLICIT'] = round(scored_tweets['SEXUALLY_EXPLICIT'] * 10, 0)

In [36]:
scored_tweets.head(5)

,author,subreddit,body,TOXICITY,PROFANITY,SEXUALLY_EXPLICIT,processed
0,Frantic_Mantid,awwnverts,Those eyes! I don't think I've ever seen a spi...,2.0,1.0,3.0,those ey i dont think iv ever seen spider ey l...
1,I_LICK_PUPPIES,longboarding,I didn't even catch on... Lmao god damn it.,8.0,9.0,2.0,i didnt even catch lmao god damn
2,PCMRwill0956,pcmasterrace,I'll take it\n,1.0,0.0,1.0,ill
3,shermont,RocketLeagueExchange,What's ur gt il send a friend req... n il let ...,5.0,4.0,3.0,what ur gt send friend req let kno still moro
4,PandaBearShenyu,ffxiv,You.... should not be losing stacks to ES even...,3.0,2.0,3.0,you lose stack es even run back forth without ...


In [37]:
#Confirms range of values for labels
scored_tweets['TOXICITY'].value_counts()

1.0     56
2.0     14
0.0     13
4.0      4
9.0      4
3.0      3
8.0      2
7.0      2
5.0      1
10.0     1
Name: TOXICITY, dtype: int64

## Models

In [223]:
import pickle

In [224]:
!pip install --upgrade xgboost

Requirement already up-to-date: xgboost in c:\programdata\anaconda3\lib\site-packages (1.5.1)


In [38]:
def split_data(df=scored_tweets, feature='processed', label='TOXICITY'):
    from sklearn.model_selection import train_test_split
    
    train_data, test_data = train_test_split(df, test_size=0.25, random_state=7)
    X_train, X_test = np.asarray(train_data['processed']), np.asarray(test_data['processed'])
    y_train, y_test = np.asarray(train_data[label]), np.asarray(test_data[label])
    
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = split_data(df=scored_tweets, feature='processed', label='TOXICITY')

In [39]:
def TFIDF(X_train, X_test):
    from sklearn.feature_extraction.text import TfidfVectorizer

    vectorizer = TfidfVectorizer()
    train_tfidf = vectorizer.fit_transform(X_train)
    test_tfidf = vectorizer.transform(X_test)
    
    return train_tfidf, test_tfidf, vectorizer

train_tfidf, test_tfidf, vectorizer = TFIDF(X_train, X_test)

In [227]:
pickle.dump(vectorizer, open('PerspectiveAPI_vectorizer.pickle', 'wb'))

In [40]:
test_tfidf

<25x655 sparse matrix of type '<class 'numpy.float64'>'
	with 117 stored elements in Compressed Sparse Row format>

In [41]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer


X_train, X_test, y_train, y_test = split_data(label='PROFANITY')

tweet = 'fuck'
processed_punc = re.sub('[^@a-zA-Z ]', '', tweet)  # remove punctuation
processed = re.sub('https\S+|@\S+|\b\w{,1}\b', '', processed_punc)  # remove https, usernames, len(word) < 1
words = word_tokenize(processed)
tokens_without_sw = [word for word in words if not word in stopwords.words()]
sentence = ' '.join(tokens_without_sw)
stemmed_sentence = p.stem_sentence(sentence)
t = stemmed_sentence

print(t)
vectorizer = TfidfVectorizer()
train_tfidf = vectorizer.fit_transform(X_train)
test_tfidf = vectorizer.transform([t])
                                 

model = XGBClassifier()
model.fit(train_tfidf, y_train)

y_pred = model.predict(test_tfidf)
y_pred

fuck


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:31:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


array([0.])

In [42]:
def XGBoost_Classifier(feature_category):
    from xgboost import XGBClassifier
    from sklearn.metrics import accuracy_score
    
    X_train, X_test, y_train, y_test = split_data(label=feature_category)
    train_tfidf, test_tfidf, _ = TFIDF(X_train, X_test)
    
    model = XGBClassifier()
    model.fit(train_tfidf, y_train)
    
    y_pred = model.predict(test_tfidf)
    acc = accuracy_score(y_pred, y_test)
    
    return (acc, model)

In [43]:
toxicity_results = XGBoost_Classifier('TOXICITY')
toxicity_results

[18:31:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


(0.44,
 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
               gamma=0, gpu_id=-1, importance_type=None,
               interaction_constraints='', learning_rate=0.300000012,
               max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
               monotone_constraints='()', n_estimators=100, n_jobs=8,
               num_parallel_tree=1, objective='multi:softprob', predictor='auto',
               random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
               subsample=1, tree_method='exact', validate_parameters=1,
               verbosity=None))

In [232]:
pickle.dump(toxicity_results[1], open('toxicity_xgb.pickle', 'wb'))

In [44]:
profanity_results = XGBoost_Classifier('PROFANITY')
profanity_results

[18:31:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


(0.48,
 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
               gamma=0, gpu_id=-1, importance_type=None,
               interaction_constraints='', learning_rate=0.300000012,
               max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
               monotone_constraints='()', n_estimators=100, n_jobs=8,
               num_parallel_tree=1, objective='multi:softprob', predictor='auto',
               random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
               subsample=1, tree_method='exact', validate_parameters=1,
               verbosity=None))

In [234]:
pickle.dump(profanity_results[1], open('profanity_xgb.pickle', 'wb'))

In [45]:
sexually_explicit_results = XGBoost_Classifier('SEXUALLY_EXPLICIT')
sexually_explicit_results

[18:31:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


(0.48,
 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
               gamma=0, gpu_id=-1, importance_type=None,
               interaction_constraints='', learning_rate=0.300000012,
               max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
               monotone_constraints='()', n_estimators=100, n_jobs=8,
               num_parallel_tree=1, objective='multi:softprob', predictor='auto',
               random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
               subsample=1, tree_method='exact', validate_parameters=1,
               verbosity=None))

In [236]:
pickle.dump(sexually_explicit_results[1], open('sexually_explicit_xgb.pickle', 'wb'))